In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

In [3]:
df = pd.read_csv('revs_scored_jan.tsv', sep='\t', header=0)

In [4]:
days = df.rev_timestamp.map(lambda ts: datetime.utcfromtimestamp(ts).day)
df['day'] = days

In [6]:
df_bots = df[df.day<22]
df_bots = df_bots[df_bots.user_is_bot]

df_reverted_bots = df_bots[df_bots.is_reverted]
df_reverted_bots = df_reverted_bots[df_reverted_bots.seconds_to_revert.astype('str').astype('int')<86400]
df_reverted_bots.revert_id = df_reverted_bots.revert_id.astype('int')

df_reverters = df[df.is_revert]

In [7]:
# merging two datafraemes to create a table of nonhuman and human reverters that reverted a bot

df_reverted_bots = pd.merge(df_reverted_bots, df_reverters, 
             how='inner', 
             left_on='revert_id', 
             right_on='rev_id', 
             suffixes=('', '_reverter')
)

In [10]:
df_reverted_bots = df_reverted_bots[['rev_id','user_text','revert_id','user_text_reverter', 'is_self_revert_reverter','user_is_bot_reverter', 'page_namespace','day']]
df_reverted_bots = df_reverted_bots.rename(columns={'is_self_revert_reverter': 'self_reverted', 'user_is_bot_reverter': 'reverter_is_bot'})

In [9]:
# current variable summary:
#   df_bots = all edits made by bots
#   df_reverters = all reverting edits in dataset
#   df_reverted_bots = all revisions by bots which were reverted (by humans or bots), with reverting user info merged